In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import statsmodels.api as sm
import plotly.express as px
import random
import functools
from datetime_truncate import truncate
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy.signal import periodogram

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from deap import base, creator, tools, algorithms
from datetime import datetime
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA

from statsmodels.tsa.arima.model import ARIMA
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit

In [2]:
df_magna = pd.read_pickle('base_magna_final.pkl')
df_premium = pd.read_pickle('base_premium_final.pkl')
df_diesel = pd.read_pickle('base_diesel_final.pkl')

In [11]:
#Calculando modelo ARIMA
model_auto_arima = auto_arima(df_magna['volumen_despachado'], seasonal=True, trace=True)
model_auto_arima.summary()



Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=23.97 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=156246.667, Time=0.41 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=156218.205, Time=0.50 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=156214.266, Time=1.64 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=156244.667, Time=0.18 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=17.61 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=156172.566, Time=2.41 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=27.19 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=156161.174, Time=6.56 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=30.30 sec
 ARIMA(0,1,4)(0,0,0)[0] intercept   : AIC=156096.026, Time=8.04 sec
 ARIMA(1,1,4)(0,0,0)[0] intercept   : AIC=inf, Time=51.98 sec
 ARIMA(0,1,5)(0,0,0)[0] intercept   : AIC=inf, Time=44.48 sec
 ARIMA(1,1,5)(0,0,0)[0] intercept   : AIC=inf, Time=59.17 sec
 ARIMA(0,1,4)(0,0,0)[0]             : AIC=15609

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                11664
Model:               SARIMAX(0, 1, 4)   Log Likelihood              -78042.013
Date:                Sun, 26 May 2024   AIC                         156094.026
Time:                        06:28:31   BIC                         156130.847
Sample:                             0   HQIC                        156106.395
                              - 11664                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -0.0728      0.005    -13.982      0.000      -0.083      -0.063
ma.L2         -0.0769      0.008     -9.533      0.000      -0.093      -0.061
ma.L3         -0.0555      0.009     -6.127      0.000      -0.073      -0.038
ma.L4         -0.0961      0.010    -10.074      0.000      -0.115      -0.077
sigma2      3.789e+04    262.787    144.169      0.000    3.74e+04    3.84e+04
===================================================================================
Ljung-Box (L1) (Q):                   0.67   Jarque-Bera (JB):             18436.43
Prob(Q):                              0.41   Prob(JB):                         0.00
Heteroskedasticity (H):               0.44   Skew:                             0.99
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.83
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

Los valores AIC para el modelo óptimo ARIMA de volumen despachado son demasiado altos (oscilan en torno a 150 mil). De esta manera, un modelo ARIMA no seria el adecuado para este tipo de datos de volumen despachado MAGNA.

In [42]:
def convert_to_unix_time(date):
    return int((date - datetime(1970, 1, 1)).total_seconds())


def feature_engineering(df_interest):
    df_interest['hour_sin'] = np.sin(2 * np.pi * df_interest['hour']/23.0)
    df_interest['hour_cos'] = np.cos(2 * np.pi * df_interest['hour']/23.0)
    df_interest['day_sin'] = np.sin(2 * np.pi * df_interest['dow_n']/6.0)
    df_interest['day_cos'] = np.cos(2 * np.pi * df_interest['dow_n']/6.0)
    df_interest['month_sin'] = np.sin(2 * np.pi * df_interest['month']/11.0)
    df_interest['month_cos'] = np.cos(2 * np.pi * df_interest['month']/11.0)
    df_interest['unix_time'] = df_interest['sale_timeStamp'].apply(convert_to_unix_time)
    df_interest.drop(['sale_timeStamp'], axis = 1, inplace = True)
    return df_interest


def setup_models(df_interest, model_features, model_target):
    alt_df = feature_engineering(df_interest[model_features].copy())
    target_df = df_interest[model_target].copy()

    new_df = pd.concat([alt_df, target_df], axis=1)

    train_percentage = 0.7
    slice_point = int(new_df.shape[0] * train_percentage)

    train_df = new_df.iloc[:slice_point]
    test_df = new_df.iloc[slice_point:]

    scaler = StandardScaler()

    X_train = scaler.fit_transform(train_df[alt_df.columns])
    X_test = scaler.transform(test_df[alt_df.columns])

    y_train = train_df[model_target].values.ravel()
    y_test = test_df[model_target].values.ravel()

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred, force_finite=False)
    print(f'Demand Prediction Model MSE: {mse}, R^2: {r2}')
    return model, X_train, scaler, df_interest['precio_neto'].mean(), alt_df

In [17]:
pe_features = ['volumen_despachado', 'precio_neto']


def pricing_elasticity_prep(df_interest, features):
    new_df = df_interest[features].copy()
    type_dict = {}

    for i, feature in enumerate(features):
        type_dict[i] = df_interest[feature].dtype

    new_df.replace(0, pd.NA, inplace=True)
    new_df.dropna(inplace=True)

    for i, feature in enumerate(features):
        new_df[feature] = new_df[feature].astype(type_dict[i])

    new_df['log_price'] = np.log(new_df['precio_neto'])
    new_df['log_demand'] = np.log(new_df['volumen_despachado'])

    return new_df

In [18]:
df_magna_alt = pricing_elasticity_prep(df_magna, pe_features)
df_premium_alt = pricing_elasticity_prep(df_premium, pe_features)
df_diesel_alt = pricing_elasticity_prep(df_diesel, pe_features)


model_lr_elasticity_magna = LinearRegression()
model_lr_elasticity_magna.fit(df_magna_alt[['log_price']], df_magna_alt[['log_demand']])
price_elasticity_magna = model_lr_elasticity_magna.coef_[0]

model_lr_elasticity_premium = LinearRegression()
model_lr_elasticity_premium.fit(df_premium_alt[['log_price']], df_premium_alt[['log_demand']])
price_elasticity_premium = model_lr_elasticity_premium.coef_[0]

model_lr_elasticity_diesel = LinearRegression()
model_lr_elasticity_diesel.fit(df_diesel_alt[['log_price']], df_diesel_alt[['log_demand']])
price_elasticity_diesel = model_lr_elasticity_diesel.coef_[0]

In [21]:
#Modelo Evolucion Diferencial (para cada base de datos)
#Aqui estuve modificando el penalty y en 10000 dio algo coherente con nuestros datos
import random
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

def feature_engineering(df_interest):
    df_interest['hour_sin'] = np.sin(2 * np.pi * df_interest['hour'] / 23.0)
    df_interest['hour_cos'] = np.cos(2 * np.pi * df_interest['hour'] / 23.0)
    df_interest['day_sin'] = np.sin(2 * np.pi * df_interest['dow_n'] / 6.0)
    df_interest['day_cos'] = np.cos(2 * np.pi * df_interest['dow_n'] / 6.0)
    df_interest['month_sin'] = np.sin(2 * np.pi * df_interest['month'] / 11.0)
    df_interest['month_cos'] = np.cos(2 * np.pi * df_interest['month'] / 11.0)
    df_interest['unix_time'] = df_interest['sale_timeStamp'].apply(lambda x: x.timestamp())
    df_interest.drop(['sale_timeStamp'], axis=1, inplace=True)
    return df_interest

def setup_models(df_interest, model_features, model_target):
    alt_df = feature_engineering(df_interest[model_features].copy())
    target_df = df_interest[model_target].copy()

    new_df = pd.concat([alt_df, target_df], axis=1)

    train_percentage = 0.7
    slice_point = int(new_df.shape[0] * train_percentage)

    train_df = new_df.iloc[:slice_point]
    test_df = new_df.iloc[slice_point:]

    scaler = StandardScaler()

    X_train = scaler.fit_transform(train_df[alt_df.columns])
    X_test = scaler.transform(test_df[alt_df.columns])

    y_train = train_df[model_target].values.ravel()
    y_test = test_df[model_target].values.ravel()

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred, force_finite=False)
    print(f'Demand Prediction Model MSE: {mse}, R^2: {r2}')
    return model, X_train, scaler, df_interest['precio_neto'].mean(), alt_df

def eval_gross_margin(individual, train_data, model, scaler, current_price, elasticity, feature_names):
    try:
        price = individual[0]
        mean_values = np.mean(train_data[:, 1:], axis=0)
        data_with_price = np.insert(mean_values, 0, price)
        temp_features = pd.DataFrame([data_with_price], columns=feature_names)

        scaled_features = scaler.transform(temp_features)
        predicted_demand = model.predict(scaled_features)[0]

        cost_of_goods_sold = temp_features['costo_neto'].iloc[0]

        gross_margin = (price - cost_of_goods_sold) * predicted_demand
        penalty_factor = 10000  # Incrementar la penalización
        penalty = penalty_factor * (abs(price - current_price) / current_price) ** (-elasticity)
        gross_margin -= penalty
        return (gross_margin,)
    except Exception as e:
        print("Error in evalGrossMargin:", e)
        return (0,)

def differential_evolution(df_interest, model_features, model_target, pe_features, lower_bound_ratio, upper_bound_ratio):
    model, X_train, scaler, current_price, features_refined = setup_models(df_interest, model_features, model_target)
    elasticity = -0.69

    lower_bound = current_price * lower_bound_ratio
    upper_bound = current_price * upper_bound_ratio

    xl = np.array([lower_bound])
    xu = np.array([upper_bound])

    G = 30  # Número de generaciones
    N = 50  # Tamaño de la población
    D = 1  # Dimensión de la búsqueda (solo precio)

    F = 0.6  # Factor de escala para la mutación
    CR = 0.9  # Tasa de recombinación

    x = np.zeros((D, N))
    fitness = np.zeros(N)

    for i in range(N):
        x[:, i] = xl + (xu - xl) * np.random.rand(D)
        fitness[i] = eval_gross_margin(x[:, i], X_train, model, scaler, current_price, elasticity, features_refined.columns)[0]

    fx_plot = np.zeros(G)

    for n in range(G):
        for i in range(N):
            r1 = i
            while r1 == i:
                r1 = np.random.randint(N)

            r2 = r1
            while r2 == r1 or r2 == i:
                r2 = np.random.randint(N)

            r3 = r2
            while r3 == r2 or r3 == r1 or r3 == i:
                r3 = np.random.randint(N)

            v = x[:, r1] + F * (x[:, r2] - x[:, r3])

            u = np.zeros(D)

            for j in range(D):
                r = np.random.rand()

                if r <= CR:
                    u[j] = v[j]
                else:
                    u[j] = x[j, i]

            fitness_u = eval_gross_margin(u, X_train, model, scaler, current_price, elasticity, features_refined.columns)[0]

            if fitness_u > fitness[i]:
                x[:, i] = u
                fitness[i] = fitness_u

        fx_plot[n] = np.max(fitness)

    igb = np.argmax(fitness)

    print("Máximo global en x =", x[:, igb], ", rentabilidad =", fitness[igb])
    return x[:, igb], fitness[igb]

pe_features = ['volumen_despachado', 'precio_neto']

model_features = ['precio_neto', 'costo_neto', 'precio_brent', 'hour', 'month', 'dow_n','sale_timeStamp']
model_target = ['volumen_despachado']

optimal_price, max_gross_margin = differential_evolution(df_magna, model_features, model_target, pe_features, 0.7, 1.3)
print(f'Optimal price: ${optimal_price[0]:.2f}')
print(f'Maximum Gross Margin: ${max_gross_margin:.2f}')


Demand Prediction Model MSE: 16544.637730899, R^2: 0.7121078523185419
Máximo global en x = [17.24642578] , rentabilidad = 2075.724777670202
Optimal price: $17.25
Maximum Gross Margin: $2075.72


In [19]:
#Modelo cuando pedi que ChatGPT lo haga para cada base de datos, incluyo nuevas funciones y librerias
#sin embargo, el modelo se crea en la parte de arriba
#Es decir, utiliza la arquitectura que armo Hans con el Toolbox pero implementando el modelo de evolucion diferencial

import pandas as pd
import numpy as np
from scipy.optimize import differential_evolution
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Feature engineering
def feature_engineering(df_interest):
    df_interest['hour_sin'] = np.sin(2 * np.pi * df_interest['hour']/23.0)
    df_interest['hour_cos'] = np.cos(2 * np.pi * df_interest['hour']/23.0)
    df_interest['day_sin'] = np.sin(2 * np.pi * df_interest['dow_n']/6.0)
    df_interest['day_cos'] = np.cos(2 * np.pi * df_interest['dow_n']/6.0)
    df_interest['month_sin'] = np.sin(2 * np.pi * df_interest['month']/11.0)
    df_interest['month_cos'] = np.cos(2 * np.pi * df_interest['month']/11.0)
    df_interest['unix_time'] = df_interest['sale_timeStamp'].apply(lambda x: pd.to_datetime(x).timestamp())
    df_interest.drop(['sale_timeStamp'], axis = 1, inplace = True)
    return df_interest

# Setup models
def setup_models(df_interest, features, target):
    alt_df = feature_engineering(df_interest[features].copy())
    target_df = df_interest[target].copy()

    new_df = pd.concat([alt_df, target_df], axis = 1)

    train_percentage = 0.7
    slice_point = int(new_df.shape[0] * train_percentage)

    train_df = new_df.iloc[:slice_point]
    test_df = new_df.iloc[slice_point:]

    scaler = StandardScaler()

    X_train = scaler.fit_transform(train_df[alt_df.columns])
    X_test = scaler.transform(test_df[alt_df.columns])

    y_train = train_df[target].values.ravel()
    y_test = test_df[target].values.ravel()

    model = RandomForestRegressor(n_estimators = 100, random_state = 42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred, force_finite = False)
    print(f'Demand Prediction Model MSE: {mse}, R^2: {r2}')
    return model, X_train, scaler, df_interest['precio_neto'].mean(), alt_df

# Evaluate gross margin
def eval_gross_margin(price, train_data, model, scaler, current_price, elasticity, feature_names, penalty_factor=100):
    try:
        price = price[0]
        # Create a dataframe to simulate the scenario with the evaluated price
        mean_values = np.mean(train_data[:, 1:], axis=0)
        data_with_price = np.insert(mean_values, 0, price)
        temp_features = pd.DataFrame([data_with_price], columns = feature_names)

        # Calculate the scaled features for the model
        scaled_features = scaler.transform(temp_features)
        predicted_demand = model.predict(scaled_features)[0]

        # Extract cost of goods sold from the temporary features, ensuring it corresponds to the simulated scenario
        cost_of_goods_sold = temp_features['costo_neto'].iloc[0]

        gross_margin = (price - cost_of_goods_sold) * predicted_demand
        penalty = penalty_factor * (abs(price - current_price) / current_price) ** (-elasticity)
        gross_margin -= penalty
        return -gross_margin  # Return negative because differential_evolution minimizes the function
    except Exception as e:
        print("Error in eval_gross_margin:", e)
        return 0  # Return a default or zero fitness in case of error

# Optimize price using Differential Evolution
def optimize_price(df, features, target, elasticity, lower_bound_ratio, upper_bound_ratio, penalty_factor):
    model, X_train, scaler, current_price, features_refined = setup_models(df, features, target)

    lower_bound = current_price * lower_bound_ratio
    upper_bound = current_price * upper_bound_ratio

    bounds = [(lower_bound, upper_bound)]

    result = differential_evolution(eval_gross_margin, bounds,
                                    args=(X_train, model, scaler, current_price, elasticity, features_refined.columns, penalty_factor),
                                    strategy='best1bin', maxiter=1000, popsize=50, tol=0.01, mutation=(0.5, 1), recombination=0.7, disp=True)

    optimal_price = result.x[0]
    max_gross_margin = -result.fun

    print(f'Optimal price: ${optimal_price:.2f}')
    print(f'Maximum Gross Margin: ${max_gross_margin:.2f}')

    return optimal_price, max_gross_margin

# Example usage
model_features = ['precio_neto', 'costo_neto', 'precio_brent', 'hour', 'month', 'dow_n','sale_timeStamp']
model_target = ['volumen_despachado']

elasticity_magna = price_elasticity_magna
elasticity_premium = price_elasticity_premium
elasticity_diesel = price_elasticity_diesel

lower_bound_ratio = 0.7
upper_bound_ratio = 1.3
penalty_factor = 100

magna_price, magna_gross_margin = optimize_price(df_magna, model_features, model_target, elasticity_magna, lower_bound_ratio, upper_bound_ratio, penalty_factor)
premium_price, premium_gross_margin = optimize_price(df_premium, model_features, model_target, elasticity_premium, lower_bound_ratio, upper_bound_ratio, penalty_factor)
diesel_price, diesel_gross_margin = optimize_price(df_diesel, model_features, model_target, elasticity_diesel, lower_bound_ratio, upper_bound_ratio, penalty_factor)



Demand Prediction Model MSE: 16544.637730899, R^2: 0.7121078523185419
differential_evolution step 1: f(x)= -2653.72
differential_evolution step 2: f(x)= -2654.46
differential_evolution step 3: f(x)= -2654.46
differential_evolution step 4: f(x)= -2654.5
differential_evolution step 5: f(x)= -2654.5
differential_evolution step 6: f(x)= -2654.5
differential_evolution step 7: f(x)= -2654.5
differential_evolution step 8: f(x)= -2654.69
Polishing solution with 'L-BFGS-B'
Optimal price: $22.42
Maximum Gross Margin: $2654.72
Demand Prediction Model MSE: 2254.319061998398, R^2: 0.4500814085871191
differential_evolution step 1: f(x)= -80.6669
differential_evolution step 2: f(x)= -80.6669
differential_evolution step 3: f(x)= -80.6669
differential_evolution step 4: f(x)= -81.0441
differential_evolution step 5: f(x)= -81.0762
differential_evolution step 6: f(x)= -81.0974
differential_evolution step 7: f(x)= -81.0974
differential_evolution step 8: f(x)= -81.148
differential_evolution step 9: f(x)= -8